### 직방의 원룸 매물정보 수집
- 절차
    - 동 이름 > 위도, 경도 수집
    - 위도, 경도(점) > geohash(범위, 영역) 수집
    - geohash(영역) > 매물 아이디 수집
    - 매물 아이디 > 매물 정보 수집 

In [1]:
import requests
import pandas as pd
import geohash2

In [2]:
# 1. 동 이름 > 위도, 경도

In [3]:
addr = '용산구 한남동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
response

<Response [200]>

In [4]:
response.text[:300]

'{"success":true,"code":"200","items":[{"id":4311,"type":"address","name":"한남동","hint":"","description":"서울시 용산구 한남동","lat":37.53458023071289,"lng":127.00049591064453,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"서울시","local2":"용산구","local3":"한남동","web_level":15,"web_lat":3'

In [5]:
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.53458023071289, 127.00049591064453)

In [6]:
# 2. 위도, 경도 > geohash

In [7]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm9'

In [8]:
# 3. geohash > 매물 아이디

In [9]:
# 한줄이 너무 길 때 역슬래쉬를 치고 엔터하면 된다. 역슬래쉬 뒤에 아무 문자도 있으면 안됨. 역슬래쉬도.
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

In [10]:
response = requests.get(url)
response

<Response [200]>

In [11]:
items = response.json()['items']
len(items), items[:2]

(783,
 [{'lat': 37.52973487718673, 'lng': 126.99838537647315, 'item_id': 32680542},
  {'lat': 37.530015589074594, 'lng': 127.00178630216995, 'item_id': 32547973}])

In [12]:
ids = [item['item_id'] for item in items]
ids[:5]

[32680542, 32547973, 32472988, 32821869, 32818112]

In [13]:
# 4. 매물 아이디 > 매물 정보 

In [14]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain': "zigbang",
    'withCoalition': 'true',
    'item_ids': ids[:900] # 99몇개부터는 안가져와짐
}
response = requests.post(url, params) # params에 한글이 없어서 json.dumps() 안해도 됨
response

<Response [200]>

In [15]:
response.text[:300]

'{"items":[{"section_type":null,"item_id":32680542,"images_thumbnail":"https://ic.zigbang.com/ic/items/32680542/1.jpg","sales_type":"월세","sales_title":"월세","deposit":1000,"rent":60,"size_m2":42.98,"공급면적":{"m2":42.98,"p":"13"},"전용면적":{"m2":39.67,"p":"12"},"계약면적":null,"room_type_title":null,"floor":"2"'

In [16]:
items = response.json()['items']
df = pd.DataFrame(items)
df.head(3) # ...이 거슬린다면?

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,32680542,https://ic.zigbang.com/ic/items/32680542/1.jpg,월세,월세,1000,60,42.98,"{'m2': 42.98, 'p': '13'}","{'m2': 39.67, 'p': '12'}",...,False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-08-01T11:26:57+09:00,False
1,None,32547973,https://ic.zigbang.com/ic/items/32547973/1.jpg,전세,전세,10000,0,36.36,"{'m2': 36.36, 'p': '11'}","{'m2': 33.21, 'p': '10'}",...,False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-07-29T17:57:20+09:00,False
2,None,32472988,https://ic.zigbang.com/ic/items/32472988/1.jpg,전세,전세,23000,0,50.12,"{'m2': 50.12, 'p': '15.2'}","{'m2': 47, 'p': '14.2'}",...,False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-07-22T11:25:58+09:00,False


In [17]:
# max row, max column 설정
pd.options.display.max_columns # 20

20

In [18]:
pd.options.display.max_columns = 50
df.head(3) # ...이 다 보인다~

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,32680542,https://ic.zigbang.com/ic/items/32680542/1.jpg,월세,월세,1000,60,42.98,"{'m2': 42.98, 'p': '13'}","{'m2': 39.67, 'p': '12'}",None,None,2,2,2,용산구 보광동 월세.주택(2층.방2)1.000/60만원,None,04,용산구 보광동,"{'lat': 37.52924586722087, 'lng': 126.99904228...",False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-08-01T11:26:57+09:00,False
1,None,32547973,https://ic.zigbang.com/ic/items/32547973/1.jpg,전세,전세,10000,0,36.36,"{'m2': 36.36, 'p': '11'}","{'m2': 33.21, 'p': '10'}",None,None,2,2,2,용산구 보광동 전세.주택(2층.방2)1억원,None,04,용산구 보광동,"{'lat': 37.530019714104526, 'lng': 127.0011643...",False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-07-29T17:57:20+09:00,False
2,None,32472988,https://ic.zigbang.com/ic/items/32472988/1.jpg,전세,전세,23000,0,50.12,"{'m2': 50.12, 'p': '15.2'}","{'m2': 47, 'p': '14.2'}",None,None,2,2,3,올수리된 꺠끗한 주차가능 방2개 전세,None,04,용산구 보광동,"{'lat': 37.52949746741094, 'lng': 126.99975339...",False,True,빌라,[],서울시 용산구 보광동,None,None,0,2022-07-22T11:25:58+09:00,False


In [21]:
columns=['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'address1', 'manage_cost']
df = pd.DataFrame(items)[columns]
df.head(5)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32680542,월세,1000,60,42.98,서울시 용산구 보광동,0
1,32547973,전세,10000,0,36.36,서울시 용산구 보광동,0
2,32472988,전세,23000,0,50.12,서울시 용산구 보광동,0
3,32821869,월세,1000,85,51.00,서울시 용산구 보광동,0
4,32818112,월세,500,40,46.28,서울시 용산구 보광동,0


In [22]:
# 함수 만들기
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain': "zigbang",
        'withCoalition': 'true',
        'item_ids': ids[:900] # 990몇개부터는 안가져와짐
    }
    response = requests.post(url, params)
    items = response.json()['items']
    columns=['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'address1', 'manage_cost']
    df = pd.DataFrame(items)[columns]
    return df

In [25]:
addr = '마포구 합정동'
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
743,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [29]:
# 합정동만 보고 싶다면?
df_filtered = df[df['address1'].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [ ]:
# 모듈 파일 만들기

In [30]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7C6C-FC35

 C:\Users\User\Downloads\AIVLE SCHOOL\웹크롤링\day2 디렉터리

2022-08-04  오후 01:30    <DIR>          .
2022-08-04  오후 01:30    <DIR>          ..
2022-08-04  오전 11:08    <DIR>          .ipynb_checkpoints
2022-08-04  오전 11:03            20,775 01_html.ipynb
2022-08-04  오후 01:30            36,624 01_zigbang.ipynb
2022-08-04  오전 11:03             7,617 02_css_selector.ipynb
2022-08-04  오전 11:04               920 02_kakao_api_실습.ipynb
2022-08-04  오전 11:03            21,109 03_naver_relational_keywords.ipynb
2022-08-04  오전 11:03           250,909 04_gmarket.ipynb
2022-08-04  오전 11:02            54,793 naver_api_2.ipynb
               7개 파일             392,747 바이트
               3개 디렉터리  133,836,169,216 바이트 남음


In [ ]:
# %%writefile zigbang.py 위에 아무 글씨도 있으면 안됨. 주석도.

In [32]:
%%writefile zigbang.py

import requests
import pandas as pd
import geohash2

def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['item_id'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain': "zigbang",
        'withCoalition': 'true',
        'item_ids': ids[:900] # 990몇개부터는 안가져와짐
    }
    response = requests.post(url, params)
    items = response.json()['items']
    columns=['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'address1', 'manage_cost']
    df = pd.DataFrame(items)[columns]
    return df

Writing zigbang.py


In [33]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7C6C-FC35

 C:\Users\User\Downloads\AIVLE SCHOOL\웹크롤링\day2 디렉터리

2022-08-04  오후 01:33    <DIR>          .
2022-08-04  오후 01:33    <DIR>          ..
2022-08-04  오전 11:08    <DIR>          .ipynb_checkpoints
2022-08-04  오전 11:03            20,775 01_html.ipynb
2022-08-04  오후 01:32            39,450 01_zigbang.ipynb
2022-08-04  오전 11:03             7,617 02_css_selector.ipynb
2022-08-04  오전 11:04               920 02_kakao_api_실습.ipynb
2022-08-04  오전 11:03            21,109 03_naver_relational_keywords.ipynb
2022-08-04  오전 11:03           250,909 04_gmarket.ipynb
2022-08-04  오전 11:02            54,793 naver_api_2.ipynb
2022-08-04  오후 01:33             1,095 zigbang.py
               8개 파일             396,668 바이트
               3개 디렉터리  133,836,005,376 바이트 남음


In [34]:
import zigbang as zb

In [35]:
df = zb.oneroom('망원동')
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
744,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
745,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [37]:
%whos
# 변수 목록

Variable      Type         Data/Info
------------------------------------
addr          str          마포구 합정동
columns       list         n=7
data          dict         n=13
df            DataFrame          item_id sales_type <...>n\n[746 rows x 7 columns]
df_filtered   DataFrame         item_id sales_type  <...>n\n[100 rows x 7 columns]
geohash       str          wydm9
geohash2      module       <module 'geohash2' from '<...>\\geohash2\\__init__.py'>
ids           list         n=783
items         list         n=783
lat           float        37.53458023071289
lng           float        127.00049591064453
oneroom       function     <function oneroom at 0x000001621BBC3D30>
params        dict         n=3
pd            module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
requests      module       <module 'requests' from '<...>\\requests\\__init__.py'>
response      Response     <Response [200]>
url           str          https://apis.zigbang.com/v2/items/list
zb            

In [38]:
%reset
# 변수 초기화

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [39]:
%whos

Interactive namespace is empty.
